In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torch.nn.functional as F

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
      x = torch.flatten(x, 1)
      x = torch.sigmoid(self.fc1(x))
      x = torch.sigmoid(self.fc2(x))
      x = self.fc3(x)
      return x

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [11]:
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 408964216.80it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 12151238.22it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 140807301.01it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11906580.48it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [12]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [13]:
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

Epoch 0 [0/60000 (0%)]	Loss: 2.395016
Epoch 0 [6400/60000 (11%)]	Loss: 2.361556
Epoch 0 [12800/60000 (21%)]	Loss: 1.700016
Epoch 0 [19200/60000 (32%)]	Loss: 1.175943
Epoch 0 [25600/60000 (43%)]	Loss: 0.757836
Epoch 0 [32000/60000 (53%)]	Loss: 0.582756
Epoch 0 [38400/60000 (64%)]	Loss: 0.601076
Epoch 0 [44800/60000 (75%)]	Loss: 0.425963
Epoch 0 [51200/60000 (85%)]	Loss: 0.545177
Epoch 0 [57600/60000 (96%)]	Loss: 0.326864
Epoch 1 [0/60000 (0%)]	Loss: 0.240026
Epoch 1 [6400/60000 (11%)]	Loss: 0.410697
Epoch 1 [12800/60000 (21%)]	Loss: 0.315686
Epoch 1 [19200/60000 (32%)]	Loss: 0.443828
Epoch 1 [25600/60000 (43%)]	Loss: 0.394981
Epoch 1 [32000/60000 (53%)]	Loss: 0.251231
Epoch 1 [38400/60000 (64%)]	Loss: 0.210354
Epoch 1 [44800/60000 (75%)]	Loss: 0.668271
Epoch 1 [51200/60000 (85%)]	Loss: 0.414646
Epoch 1 [57600/60000 (96%)]	Loss: 0.291394
Epoch 2 [0/60000 (0%)]	Loss: 0.363373
Epoch 2 [6400/60000 (11%)]	Loss: 0.406720
Epoch 2 [12800/60000 (21%)]	Loss: 0.240814
Epoch 2 [19200/60000 (32%)]	L

In [14]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()

test_loss /= len(test_loader.dataset)
print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)')


Test set: Average loss: 0.0001, Accuracy: 9678/10000 (97%)
